In [1]:
!pip install pennylane
!pip install pennylane-cirq
!pip install qiskit-aqua
!pip install pennylane-qiskit
!pip install torch

     |████████████████████████████████| 286kB 2.8MB/s 
     |████████████████████████████████| 1.4MB 4.6MB/s 
     |████████████████████████████████| 1.9MB 3.4MB/s 
     |████████████████████████████████| 583kB 17.4MB/s 
     |████████████████████████████████| 6.7MB 33.4MB/s 
     |████████████████████████████████| 26.0MB 1.7MB/s 
     |████████████████████████████████| 184kB 44.3MB/s 
     |████████████████████████████████| 5.8MB 48.4MB/s 
     |████████████████████████████████| 962kB 36.8MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
  Created wheel for dlx: filename=dlx-1.0.4-cp36-none-any.whl size=5712 sha256=4583b737428861409310b9e77edd5815992261bcb92ca1c9fab25d8b85eb0f4c
  Stored in directory: /root/.cache/pip/wheels/bb/ba/15/fdd0deb104df3254912998150ba9245668db06b00af5912d1a
  Created wheel for docplex: filename=docplex-2.15.194-cp36-none-any.whl size=645115 sha256=66c1b10682d2fde5948baea4daec7a0a7f4d7ad131d9d22e

In [1]:
import pennylane as qml
import matplotlib.pyplot as plt
from qiskit.aqua.components.uncertainty_models import UnivariateVariationalDistribution, NormalDistribution
from pennylane.templates.embeddings import AmplitudeEmbedding
from pennylane.templates.layers import StronglyEntanglingLayers
from pennylane.templates.state_preparations import MottonenStatePreparation
from pennylane.init import strong_ent_layers_uniform
import pennylane.numpy as np
import autograd

In [8]:
dev1 = qml.device('cirq.simulator', wires=3)
dev2 = qml.device('qiskit.aer',wires=3)

In [10]:
def encoding(x = None):
  for i in range(4):
    qml.RY(x[i],wires = [i])
  for i in range(3):
    qml.CNOT(wires = [i,i+1])
  qml.CNOT(wires = [3,0])
  for i in range(4):
    qml.RZ(x[i+4],wires = [i])
  for i in range(2):
    qml.CNOT(wires = [i,i+2])
  qml.CNOT(wires = [2,0])
  qml.CNOT(wires = [3,1])



In [11]:
@qml.qnode(dev2)
def generator(w):
  StronglyEntanglingLayers(w, wires=[0,1,2])
  return qml.probs(wires=range(3))

In [12]:
@qml.qnode(dev1)
def discriminator(w,x = None):
  MottonenStatePreparation(x, wires = [0,1,2])
  StronglyEntanglingLayers(w, wires=[0,1,2])
  return qml.expval(qml.PauliZ(wires = 0))


In [16]:
np.random.seed(0)

def real_true(disc_weights):
  true_prob = np.array([0,0,0,0,0,0,0,1])
  disc_output = (discriminator(disc_weights,x = true_prob) + 1)/2
  return disc_output  

def fake_true(gen_weights, disc_weights):
  prob = generator(gen_weights)
  prob = np.sqrt(prob)
  disc_output = (discriminator(disc_weights, x=prob) + 1)/2
  return disc_output

def disc_cost(disc_weights):
  cost = fake_true(gen_weights, disc_weights) - real_true(disc_weights)
  return cost

def gen_cost(gen_weights):
  return -fake_true(gen_weights, disc_weights)



In [17]:
gen_weights = strong_ent_layers_uniform(n_layers=3, n_wires=3)
disc_weights = strong_ent_layers_uniform(n_layers=3, n_wires=3)

disc_weights.shape

(3, 3, 3)

In [20]:
opt = qml.AdagradOptimizer(stepsize=0.1, eps=1e-08)
# set the number of steps
steps = 100
# set the initial parameter values

for i in range(steps):
    # update the circuit parameters
    disc_weights = opt.step(disc_cost, disc_weights)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, disc_cost(disc_weights)))

print("Optimized rotation angles: {}".format(disc_weights))

Cost after step     5: -0.9675666
Cost after step    10: -0.9770840
Cost after step    15: -0.9808902
Cost after step    20: -0.9886423
Cost after step    25: -0.9897671
Cost after step    30: -0.9879165
Cost after step    35: -0.9907352
Cost after step    40: -0.9887427
Cost after step    45: -0.9901944
Cost after step    50: -0.9889970
Cost after step    55: -0.9910394
Cost after step    60: -0.9925185
Cost after step    65: -0.9909426
Cost after step    70: -0.9905737
Cost after step    75: -0.9841330
Cost after step    80: -0.9914705
Cost after step    85: -0.9920890
Cost after step    90: -0.9922381
Cost after step    95: -0.9883683
Cost after step   100: -0.9874150
Optimized rotation angles: [[[ 6.18575555  3.09130443  2.17848049]
  [ 1.45460907  4.51171984  3.01949178]
  [ 2.85455363  0.31893224  4.04511225]]

 [[ 4.20071753  3.2106813   5.49471003]
  [ 4.70325078  3.03829138  2.58614494]
  [ 4.07204302 -0.06653418  3.84199546]]

 [[ 4.21484302  1.32144701  0.81006782]
  [ 1.510

In [21]:
opt = qml.optimize.RotosolveOptimizer()
# set the number of steps
steps = 50
# set the initial parameter values

for i in range(steps):
    # update the circuit parameters
    gen_weights = opt.step(gen_cost, gen_weights)
    #print(params, type(params))
    #gen_weights = params

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, gen_cost(gen_weights)))

print("Optimized rotation angles: {}".format(gen_weights))

Cost after step     5: -0.9394419
Cost after step    10: -0.9344345
Cost after step    15: -0.9572276
Cost after step    20: -0.9450917
Cost after step    25: -0.9573423
Cost after step    30: -0.9234007
Cost after step    35: -0.9451738
Cost after step    40: -0.9425347
Cost after step    45: -0.9290797
Cost after step    50: -0.9680209
Optimized rotation angles: [[[-0.27159776  2.03933555 -1.6541293 ]
  [ 0.36354742  2.31376844 -2.39035047]
  [ 1.19344581 -1.97049972  2.14433902]]

 [[ 2.40456466 -1.46444568  2.13785844]
  [-2.52917735 -2.61070361  0.20345007]
  [-2.54508104  0.2305002  -2.06627153]]

 [[-2.1559029  -2.28266751  2.03261127]
  [-2.70613521 -2.20103475  1.41482838]
  [ 2.23249214 -2.31357872  2.84899847]]]


At the joint optimum, both of real and fake data should be classified as real.




In [33]:
print("Probability of real data classified as real: ", real_true(disc_weights))


Probability of real data classified as real:  0.9949092241004109


In [34]:
print("Probability of fake data classified as real: ", fake_true(gen_weights, disc_weights))

Probability of fake data classified as real:  0.9693305101245642


We define the discriminator's cost as difference between how it classifies fake and real data. At the join optimum, we see that it is much closer to zero, i.e. it classifies both of them as real.

In [29]:
disc_cost(disc_weights)

-0.02980396058410406

In [32]:
gen_cost(gen_weights)

-0.9702772051095963

In [35]:
opt = qml.AdagradOptimizer(stepsize=0.1, eps=1e-08)
# set the number of steps
steps = 100
# set the initial parameter values

for i in range(steps):
    # update the circuit parameters
    disc_weights = opt.step(disc_cost, disc_weights)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, disc_cost(disc_weights)))

print("Optimized rotation angles: {}".format(disc_weights))

Cost after step     5: -0.2079672
Cost after step    10: -0.2811905
Cost after step    15: -0.2636185
Cost after step    20: -0.2809192
Cost after step    25: -0.2790170
Cost after step    30: -0.3004659
Cost after step    35: -0.2760575
Cost after step    40: -0.2927808
Cost after step    45: -0.2908223
Cost after step    50: -0.2735720
Cost after step    55: -0.2864361
Cost after step    60: -0.2986540
Cost after step    65: -0.2918904
Cost after step    70: -0.2755597
Cost after step    75: -0.2874912
Cost after step    80: -0.2897929
Cost after step    85: -0.2846162
Cost after step    90: -0.2605368
Cost after step    95: -0.3154858
Cost after step   100: -0.2858409
Optimized rotation angles: [[[6.27261529 3.43549846 1.96944212]
  [1.40466186 4.84719107 2.49109884]
  [2.72764927 0.78627267 4.26462027]]

 [[4.10514369 3.55969276 5.6276564 ]
  [5.20340402 3.01864988 2.11446973]
  [4.71397345 0.60176476 4.40214323]]

 [[4.21442777 1.32044235 0.81006782]
  [2.11193672 1.64044069 3.582

In [37]:
opt = qml.optimize.RotosolveOptimizer()
# set the number of steps
steps = 50
# set the initial parameter values

for i in range(steps):
    # update the circuit parameters
    gen_weights = opt.step(gen_cost, gen_weights)
    #print(params, type(params))
    #gen_weights = params

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, gen_cost(gen_weights)))

print("Optimized rotation angles: {}".format(gen_weights))

Cost after step     5: -0.8787397
Cost after step    10: -0.8474563
Cost after step    15: -0.9068616
Cost after step    20: -0.8724469
Cost after step    25: -0.8665668
Cost after step    30: -0.8781463
Cost after step    35: -0.8788528
Cost after step    40: -0.8852291
Cost after step    45: -0.8932707
Cost after step    50: -0.8899779
Optimized rotation angles: [[[ 0.84692694  1.26122144  3.05768646]
  [ 3.10398148  0.03667763  2.84082819]
  [-3.00850887  0.47026106 -2.67302756]]

 [[ 2.95540828 -0.02740989 -2.88006851]
  [-1.52823093  2.72168353  0.14552281]
  [ 1.66580213  0.33713048 -2.76870724]]

 [[ 2.79604262 -1.92761117  2.30282886]
  [ 0.62801785 -0.16546618 -0.52973832]
  [ 2.5724413  -2.17532428  2.35086854]]]


In [38]:
print("Probability of real data classified as real: ", real_true(disc_weights))
print("Probability of fake data classified as real: ", fake_true(gen_weights, disc_weights))
print("Discriminator's cost", disc_cost(disc_weights))
print("Generator's cost", gen_cost(gen_weights))

Probability of real data classified as real:  0.6411762535572052
Probability of fake data classified as real:  0.8787557706236839
Discriminator's cost 0.21703577041625977
Generator's cost -0.8718947768211365


In [39]:
opt = qml.AdagradOptimizer(stepsize=0.1, eps=1e-08)
# set the number of steps
steps = 100
# set the initial parameter values

for i in range(steps):
    # update the circuit parameters
    disc_weights = opt.step(disc_cost, disc_weights)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, disc_cost(disc_weights)))

print("Optimized rotation angles: {}".format(disc_weights))

Cost after step     5: -0.4621756
Cost after step    10: -0.8743326
Cost after step    15: -0.9545222
Cost after step    20: -0.9735260
Cost after step    25: -0.9825021
Cost after step    30: -0.9888439
Cost after step    35: -0.9880031
Cost after step    40: -0.9883974
Cost after step    45: -0.9871169
Cost after step    50: -0.9890269
Cost after step    55: -0.9900055
Cost after step    60: -0.9888216
Cost after step    65: -0.9874564
Cost after step    70: -0.9905463
Cost after step    75: -0.9901802
Cost after step    80: -0.9904425
Cost after step    85: -0.9897420
Cost after step    90: -0.9883959
Cost after step    95: -0.9891541
Cost after step   100: -0.9912880
Optimized rotation angles: [[[6.08910656 3.23602785 2.61957208]
  [1.51833897 4.59295786 1.97780923]
  [2.9379148  0.648445   4.42803766]]

 [[3.26615532 3.18239586 6.5210824 ]
  [5.20172888 3.03521093 2.13221354]
  [4.7412106  0.45586384 5.07887817]]

 [[4.21505632 1.31944485 0.81006782]
  [1.43502399 1.6849067  3.582

In [40]:
opt = qml.optimize.RotosolveOptimizer()
# set the number of steps
steps = 50
# set the initial parameter values

for i in range(steps):
    # update the circuit parameters
    gen_weights = opt.step(gen_cost, gen_weights)
    #print(params, type(params))
    #gen_weights = params

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, gen_cost(gen_weights)))

print("Optimized rotation angles: {}".format(gen_weights))

Cost after step     5: -0.9253006
Cost after step    10: -0.9403524
Cost after step    15: -0.9099847
Cost after step    20: -0.9261862
Cost after step    25: -0.9765887
Cost after step    30: -0.9510366
Cost after step    35: -0.9421568
Cost after step    40: -0.9698373
Cost after step    45: -0.9413295
Cost after step    50: -0.9679104
Optimized rotation angles: [[[-3.07086535 -1.72808539  2.57439027]
  [ 1.71888053  0.40383659 -0.08101301]
  [ 0.0242624  -1.87995541 -2.34665405]]

 [[-2.2287843   0.7631473   2.95515715]
  [-2.22803797  3.11246191 -1.78512049]
  [ 2.25516267 -2.5430845  -2.46743891]]

 [[ 2.40209006 -2.08934234  1.22405792]
  [-3.0128833  -2.75696487  1.37742984]
  [ 2.89951072 -2.17202077 -0.18281621]]]


In [41]:
print("Probability of real data classified as real: ", real_true(disc_weights))
print("Probability of fake data classified as real: ", fake_true(gen_weights, disc_weights))
print("Discriminator's cost", disc_cost(disc_weights))
print("Generator's cost", gen_cost(gen_weights))

Probability of real data classified as real:  0.995952125871554
Probability of fake data classified as real:  0.9689365578815341
Discriminator's cost -0.024314356269314885
Generator's cost -0.9696989897638559


We can continue to train discriminator and generator alternatively until both of them no longer converge, i.e. an optimal point is reached.
